In [1]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO
import pickle
import matplotlib.pyplot as plt

import environment

In [2]:
# Electricity rates
def func_price(i):
    # $/kWh
    hour = (i//6)%24
    if hour<7:
        return 0.082
    elif (hour>=7)&(hour<11):
        return 0.113
    elif (hour>=11)&(hour<17):
        return 0.17
    elif (hour>=17)&(hour<19):
        return 0.113
    else:
        return 0.082

# HVAC COP
def func_COP(To):
    return -2/25 * To + 6

In [3]:
with open('HW2_Data.pickle', 'rb') as f:
    Data = pickle.load(f)
To = Data['To'][:6*24*2] # outdoor air temperature in degC
q_solar = Data['q_solar'][:6*24*2] # absorbed solar radiation in W

with open('HW2_SS-Model.pickle', 'rb') as f:
    ss_model = pickle.load(f)
Ad = ss_model['Ad']
Bd = ss_model['Bd']
Cd = ss_model['Cd']
Dd = ss_model['Dd']

Bd_HVAC = Bd[:,:1]
Bd_dist = Bd[:,1:]
Dd_HVAC = Dd[:1]
Dd_dist = Dd[1:]

In [4]:
u_dist = np.hstack((q_solar[:,None], To[:,None]))
Price = np.array([func_price(i) for i in range(q_solar.shape[0])])
COP = np.array([func_COP(i) for i in To])

Environment = environment.SimpleEnv(Ad, Bd_HVAC, Bd_dist,
                        Cd, Dd_HVAC, Dd_dist,
                        COP, Price, u_dist)

In [ ]:
model = PPO("MultiInputPolicy", Environment, verbose=1, learning_rate=0.0001, gamma=0.999)#, n_steps=288)
model.learn(total_timesteps=1_000_000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 288      |
|    ep_rew_mean     | -190     |
| time/              |          |
|    fps             | 1700     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 288         |
|    ep_rew_mean          | -108        |
| time/                   |             |
|    fps                  | 1125        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010228768 |
|    clip_fraction        | 0.125       |
|    clip_range           | 0.2         |
|    entropy_loss   

In [ ]:
vec_env = model.get_env()
obs_history=[]
act_history=[]
obs = vec_env.reset()
obs_history.append(obs)
for i in range(288):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    obs_history.append(obs)
    act_history.append(action)

In [ ]:
result = {'obs_history':obs_history,
          'act_history':act_history}
with open('tmp.pickle', 'wb') as f:
    pickle.dump(result, f)

In [ ]:
reward